In [1]:
# Load required libraries
library(tidyverse)
library(lubridate)
library(GGally)

# Set directory
#Set working directory
setwd("~//OneDrive/git-repos/EMMA")

# Read the CSV file
data <- read_csv("isco_metadata/LCBP_RI_sample_index/RI23-IC-ICP-isotope-joined.csv")

# Filter to only include Site == "Wade" and Type in specified categories
data <- data %>%
  filter(Site == "Wade", Type %in% c("Grab", "Grab/Isco", "Baseflow"))

# List of solute columns
solutes <- c('Fe_mg_L', 'Mn_mg_L', 'Cu_mg_L', 'Zn_mg_L', 'Si_mg_L', 'K_mg_L', 
             'P_mg_L', 'Mg_mg_L', 'Na_mg_L', 'Al_mg_L', 'Ca_mg_L', 'Fl_mg_L', 
             'Cl_mg_L', 'NO2_mg_L', 'Br_mg_L', 'NO3_mg_L', 'PO4_mg_L', 'SO4_mg_L', 
             'dD', 'd18O')

# Convert Date to proper date format and extract month
data <- data %>%
  mutate(Date = mdy(Date),
         Month = month(Date))

# Define custom season colors
season_colors <- c(
  "1" = "blue", "2" = "blue", "3" = "green", "4" = "green", "5" = "green",
  "6" = "red", "7" = "red", "8" = "red", "9" = "saddlebrown", "10" = "saddlebrown",
  "11" = "blue", "12" = "blue"
)

# Add color column based on month
data <- data %>%
  mutate(SeasonColor = season_colors[as.character(Month)])

# Subset only solute columns and add color for plotting
plot_data <- data %>%
  select(all_of(solutes)) %>%
  mutate(SeasonColor = data$SeasonColor)

# Create a ggpairs plot for lower triangle with season colors
ggpairs(
  plot_data,
  columns = 1:length(solutes),
  lower = list(continuous = wrap("points", alpha = 0.5, colour = plot_data$SeasonColor)),
  upper = list(continuous = "blank"),
  diag = list(continuous = "blankDiag")
)

Warning message:
“package ‘tidyverse’ was built under R version 4.2.3”
Warning message:
“package ‘ggplot2’ was built under R version 4.2.3”
Warning message:
“package ‘tibble’ was built under R version 4.2.3”
Warning message:
“package ‘tidyr’ was built under R version 4.2.3”
Warning message:
“package ‘readr’ was built under R version 4.2.3”
Warning message:
“package ‘purrr’ was built under R version 4.2.3”
Warning message:
“package ‘dplyr’ was built under R version 4.2.3”
Warning message:
“package ‘stringr’ was built under R version 4.2.3”
Warning message:
“package ‘forcats’ was built under R version 4.2.3”
Warning message:
“package ‘lubridate’ was built under R version 4.2.3”
── Attaching core tidyverse packages ──────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ──────────